In [3]:
# Opracowane na podstawie https://minpy.readthedocs.io/en/latest/tutorial/rnn_mnist.html
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    RNN((14 * 14) => 64, tanh),
    Dense(64 => 10, identity),
)

Chain(
  Recur(
    RNNCell(196 => 64, tanh),           # 16_768 parameters
  ),
  Dense(64 => 10),                      # 650 parameters
)         # Total: 6 trainable arrays, 17_418 parameters,
          # plus 1 non-trainable, 64 parameters, summarysize 68.406 KiB.

In [9]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    Flux.reset!(model)
    ŷ = model(x[  1:196,:])
    display(x[  1:196,:])
    ŷ = model(x[197:392,:])
    ŷ = model(x[393:588,:])
    ŷ = model(x[589:end,:])
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 15e-3,
    epochs = 5,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

196×10000 Matrix{Float32}:
 0.0        0.0  0.0       0.0  …  0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0  …  0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 0.0        0.0  0.0       0.0     0.0        0.0       0.0       0.0
 ⋮                              ⋱                                 
 0.0117647  0.0  0.960784  0.0     0.0        0.992157  0.960784  0.0
 0.0        0.0  0.988235  0.0     0.0196078  0.992157  0.427451  0.0
 0.0        0.0  0.290196  0.0     0.403922   0.988235  0.0       

loss_and_accuracy(net, test_data) = nothing


In [11]:
using ProgressMeter

for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        Flux.reset!(net)
        grads = Flux.gradient(model -> let
                ŷ = model(x[  1:196,:])
                ŷ = model(x[197:392,:])
                ŷ = model(x[393:588,:])
                ŷ = model(x[589:end,:])
                Flux.logitcrossentropy(ŷ, y)
            end, net)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

10×60000 Matrix{Float32}:
 -0.213655   -0.332257    0.0995641  …  -0.73018     0.208137    0.0638069
  0.133647   -0.361153    0.302384      -0.36615     0.176838   -0.478121
 -0.199316   -0.822529   -0.426453      -0.580882    0.0509286  -0.96359
 -0.273503   -0.0808809  -0.43193        0.0125504  -0.0694752   0.471783
  0.52688    -0.301246    0.214178      -0.720722    0.28851    -0.296031
 -0.123806   -0.0633075   0.131075   …  -0.712444    0.198246    0.161361
 -0.262311   -0.384467   -0.313968      -0.0554412  -0.13532    -0.799805
  0.250082    1.53114    -0.0790297      2.16761     0.028474    1.38873
 -0.330798   -0.0832345  -0.467489      -0.406275   -0.36744     0.434755
  0.0197034   0.570053   -0.0917165      1.00348    -0.109199    0.362981

10×10000 Matrix{Float32}:
 -0.291492    -0.312734   -0.33603    …  -0.740249  -0.126825  -0.348973
 -0.00408123  -0.0189533   0.0530193     -0.452925   0.282625  -0.0890535
 -0.651393    -0.168041   -0.207269      -0.460895  -0.140149   0.166949
 -0.20202     -0.0866933  -0.194527       0.509621  -0.06125    0.330517
 -0.430193    -0.806245    0.660921      -0.726146   0.241251  -0.109103
 -0.151222    -0.146755   -0.0665411  …  -0.553347   0.145752  -0.120537
 -0.324633    -0.662058   -0.274155      -0.680688  -0.129487  -0.19254
  1.76352      1.11444     0.421602       0.994752   0.126187   0.144873
 -0.307364    -0.311092   -0.261874       0.334594  -0.282712  -0.101142
  0.671077     0.966145    0.141297       0.944174  -0.108451   0.0743774

 11.171415 seconds (22.20 M allocations: 4.006 GiB, 1.55% gc time, 93.73% compilation time)


┌ Info: 1
│   acc = 89.74
│   test_acc = 90.27
└ @ Main /home/maciek/Templates/AutomaticDifferention/addons/rnn.ipynb:18


10×60000 Matrix{Float32}:
 -0.200779   -0.553401   -0.225164   …  -0.500487   -0.219185   -0.624833
 -0.154705   -0.0121608   0.351074      -0.214506    0.247815   -0.314294
 -0.345252   -0.403799   -0.0847212     -0.854151   -0.130951   -0.898983
 -0.280145   -0.343904   -0.0643481     -0.316027    0.044967   -0.316845
 -0.0429397  -0.079158    0.244391      -0.14866     0.446781   -0.546441
  0.0860017   0.0244702   0.256749   …  -0.0225209   0.431617    0.0588507
 -0.993717   -0.365934   -0.173278      -0.683438   -0.28239    -1.20909
  1.1833      1.62554     0.190716       1.57744     0.490149    1.95773
 -0.161488   -0.527498   -0.401482      -0.434034   -0.274072   -0.591619
  0.899614    0.889598   -0.13617        0.688616    0.0765667   1.83453

...

┌ Info: 4
│   acc = 93.9
│   test_acc = 93.88
└ @ Main /home/maciek/Templates/AutomaticDifferention/addons/rnn.ipynb:18


10×60000 Matrix{Float32}:
 -0.432462   -0.620048   -0.474095   …  -0.305789   -0.521863   -1.34476
  0.480387    0.245895   -0.115739       0.276305    0.408333   -1.13653
  0.0804422  -0.0407934  -0.887538      -0.0156336   0.0826199  -0.835716
 -0.0700365   0.272757    0.0370414      0.186325   -0.017948    0.75463
  0.160817    0.126342   -0.478252       0.20585     0.267944   -0.990837
  0.473615    0.137719    0.315655   …   0.569842    0.49055    -0.377923
 -0.255719   -0.520423   -1.26891       -0.496233   -0.251506   -0.52922
  0.325616    0.756927    1.08231        0.358873    0.513236    3.19574
 -0.617947   -0.446685   -0.523259      -0.640891   -0.574454   -0.869935
 -0.212122    0.660088    1.35033       -0.01321    -0.0306596   2.37239

10×10000 Matrix{Float32}:
 -0.43869    -1.32467   -0.48793    …  -1.86499   -1.39773    -1.42661
 -0.0573533  -0.349788   0.226358      -0.997718   0.158519   -0.697928
 -0.661876   -1.6127    -0.304049      -1.88248   -0.635982   -0.787849
  0.163891   -1.11236   -0.409118       1.19799    0.292961    0.566025
 -0.641763   -0.248268   0.13731       -2.17349    0.525081   -0.562812
  0.388677    0.208352   0.0405928  …   0.18522   -0.0356763  -0.0317152
 -1.15554    -0.775594  -0.31554       -1.34807    0.195189   -0.202108
  1.20535     1.55674    0.991164       4.68596    0.860269    3.19678
 -0.331855   -0.71974   -0.441225      -0.582652  -0.77032    -0.919591
  1.23635     2.23694    0.178571       2.87533    1.62226     2.23182

  0.715945 seconds (541.34 k allocations: 2.630 GiB, 31.78% gc time)


┌ Info: 5
│   acc = 94.49
│   test_acc = 94.55
└ @ Main /home/maciek/Templates/AutomaticDifferention/addons/rnn.ipynb:18


In [4]:
Flux.reset!(net)
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1[  1:196,:])
y1hat = net(x1[197:392,:])
y1hat = net(x1[393:588,:])
y1hat = net(x1[589:end,:])
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [5 5]
loss_and_accuracy(net, train_data) = (loss = 0.19652821f0, acc = 94.38, split = :train)
